# Diffuse scattering integration with *mdx2*

In this tutorial, we will:

- Integrate diffuse scattering on a fractional Miller index grid
- Correct for geometry and background scattering

Requirements:
- `geometry.nxs` from the *mdx2* geometry tutorial
- `data.nxs`, `bkg_data_binned.nxs`, `mask.nxs` from the *mdx2* data tutorial

Imports (run this before you do anything else)

In [1]:
# for loading mdx2 objects in python
from mdx2.utils import loadobj

# for display purposes, and for calculating statistics
import numpy as np
import pandas as pd

## Integrate diffuse scattering on a fractional Miller index grid

### *mdx2.integrate*

In [3]:
!mdx2.integrate --help

usage: mdx2.integrate [-h] [--mask MASK] [--subdivide N N N]
                      [--max_spread DEGREES] [--outfile OUTFILE]
                      geom data

Integrate counts in an image stack on a Miller index grid

positional arguments:
  geom                  NeXus file with miller_index
  data                  NeXus file with image_series

optional arguments:
  -h, --help            show this help message and exit
  --mask MASK           NeXus file with mask (default: None)
  --subdivide N N N     subdivisions of the Miller index grid (default: [1, 1,
                        1])
  --max_spread DEGREES  maximum angular spread for binning partial
                        observations (default: 1.0)
  --outfile OUTFILE     name of the output NeXus file (default:
                        integrated.nxs)


### Parameters

The main thing we need to decide is how to divide up reciprocal space into voxels. For the tutorial, I've chosen to subdivide by 4 in each direction, meaning voxels will be centered at h = 1, 1.25, 1.5, 1.75, etc. 

This is by no means the optimal choice: the limit depends on the mosaicity, experimental geometry, and most importantly what you hope to learn from the diffuse scattering map. You may want to experiment with other subdivisions. But beware that finer maps require a lot more memory and disk space.

Integration is the slowest step in data reduction. Be patient.

In [7]:
!mdx2.integrate geometry.nxs data.nxs --mask mask.nxs --subdivide 4 4 4

Reading miller_index from geometry.nxs
  importing as MillerIndex from mdx2.geometry
Reading image_series from data.nxs
  importing as ImageSeries from mdx2.data
Looping through chunks
  binned chunk 0 from 1891042 to 11799 voxels
  binned chunk 1 from 1891704 to 16089 voxels
  binned chunk 2 from 1891701 to 18091 voxels
  binned chunk 3 from 1891522 to 16321 voxels
  binned chunk 4 from 1891069 to 12286 voxels
  binned chunk 5 from 1890751 to 12084 voxels
  binned chunk 6 from 1891722 to 16991 voxels
  binned chunk 7 from 1891538 to 19162 voxels
  binned chunk 8 from 1891323 to 17110 voxels
  binned chunk 9 from 1891159 to 12590 voxels
  binned chunk 10 from 1891123 to 11731 voxels
  binned chunk 11 from 1891316 to 16922 voxels
  binned chunk 12 from 1891533 to 19300 voxels
  binned chunk 13 from 1891388 to 17004 voxels
  binned chunk 14 from 1891623 to 12171 voxels
  binned chunk 15 from 1891288 to 10482 voxels
  binned chunk 16 from 1891766 to 15408 voxels
  binned chunk 17 from 189

In [8]:
!mdx2.tree integrated.nxs

integrated.nxs: root:NXroot
  @HDF5_Version = '1.10.6'
  @file_name = '/Users/steve/Documents/GitHub/ando-lab/erice-2...'
  @file_time = '2022-05-30T16:16:05.536262'
  @h5py_version = '3.3.0'
  @nexusformat_version = '0.7.5'
  entry:NXentry
    hkl_table:NXgroup
      @mdx2_class = 'HKLTable'
      @mdx2_module = 'mdx2.data'
      counts = int32(2047443)
      h = float32(2047443)
      ix = float32(2047443)
      iy = float32(2047443)
      k = float32(2047443)
      l = float32(2047443)
      n = float64(2047443)
      ndiv = [4 4 4]
      phi = float32(2047443)
      pixels = int32(2047443)
      seconds = float32(2047443)


### Integration statistics

Integration produces a giant table indexed by fractional h,k,l values.

In [10]:
# load the table
tab = loadobj('integrated.nxs','hkl_table')

# convert to pandas dataframe
df = tab.to_frame().set_index(['h','k','l'])

# show the first several entries
df.head()

Reading hkl_table from integrated.nxs
  importing as HKLTable from mdx2.data


counts        ix        iy    n         phi  pixels  \
h     k      l                                                            
-65.5 -16.75 -9.75       2  0.333333  1.666667  0.0  160.449997       3   
             -9.50      12  0.869565  2.304348  0.0  160.250000      23   
             -9.25      15  1.892857  1.928571  0.0  160.078568      28   
      -16.50 -9.50       5  0.000000  6.166667  0.0  160.183334       6   
             -9.25       3  0.285714  6.857143  0.0  160.050003       7   

                    seconds  
h     k      l               
-65.5 -16.75 -9.75      0.3  
             -9.50      2.3  
             -9.25      2.8  
      -16.50 -9.50      0.6  
             -9.25      0.7

## Correct for geometry and background scattering